In [1]:
import pandas as pd
import xgboost
from sklearn.ensemble import AdaBoostRegressor
from collections import defaultdict
import numpy as np
from sklearn.neural_network import MLPRegressor
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale
import matplotlib
import matplotlib.lines as mlines
import matplotlib.backends.backend_pdf
import matplotlib.transforms as mtransforms
from sklearn.ensemble import RandomForestRegressor
from collections import defaultdict
from sklearn.preprocessing import scale
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from collections import defaultdict
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeRegressor
def smape(A, F):
    if any(x==0 for x in (np.abs(A) + np.abs(F))):
        return -1
    else:
        return 100/len(A) * np.sum(2 * np.abs(F - A) / (np.abs(A) + np.abs(F)))/100

C:\Users\Alexander\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


## Print to csv

In [2]:
#The function, that will transform this to big dataframe and save to csv
def print_to_csv(y_predicted,y_true,last_rpd,predicted_rpd,method_name = "test"):
    needed_values = [(3,0),(4,0),(5,0),(6,0),(7,0),(8,0),(8,1),(8,2),(8,3),(2,4),(3,4),(4,4),(5,4),(6,4),(7,4),(8,4),(8,5),(8,7),(1,6),(2,6),(3,6),(4,6),(5,6),(6,6),(7,6),(8,6),(8,8),(0,9),(1,9),(2,9),(3,9),(4,9),(5,9),(6,9),(7,9),(8,9)]
    exp = pd.DataFrame(columns=['item','rpd','future_flag','true_value','predicted','method'])
    for it in y_predicted.keys():
        true_values = []
        predicted_values = []
        for x in needed_values:
            true_values.append(y_true[it].values[x[0]][x[1]])
            predicted_values.append(y_predicted[it][x[1]][x[0]])
        #create row of 
        row_of_prediction = [y_true[it]]
        i = 0
        for ff in range(0,4):
            for rpd in range(predicted_rpd,last_rpd+1):
                d1 = pd.DataFrame([[it,rpd,ff,true_values[i],predicted_values[i],method_name]], columns=['item','rpd','future_flag','true_value','predicted','method'])
                i+=1
                exp = exp.append(d1)
    exp = exp.set_index(["item", "rpd", "future_flag"]).sort_index()
    exp.to_csv(method_name+'.csv')
    return exp.drop(columns = ["method","predicted"]).reset_index(),exp.drop(columns = ["method","true_value"]).reset_index()

## Plotting

In [32]:
def plot_hist(plot_array,message = "",close = 1):
    pdf = matplotlib.backends.backend_pdf.PdfPages("plots/"+message+".pdf")
    
    for i in range(0,4):
        smape_res = smape_(plot_array[i][0], plot_array[i][1], mean = False)
        fig, ax = plt.subplots(figsize = (5, 6))
        plt.hist(smape_res)
        plt.axvline(smape_res.mean(), color='r', linestyle='dashed', linewidth=1)
        ax.set_xlabel('SMAPE, %. '+message+' Flag = '+str(i)+'.')
        plt.savefig('plots/Hist. '+message+' Flag = '+str(i)+'.jpg',dpi =300)
        ax.annotate('Mean SMAPE = '+str(int(smape_res.mean()))+" %",
            xy=(smape_res.mean(), ax.get_ylim()[1]), xytext=(smape_res.mean()+(ax.get_xlim()[1]-ax.get_xlim()[0])*0.07, ax.get_ylim()[1]*1.015),
            arrowprops=dict(facecolor='black', shrink=0.1),
            annotation_clip=False)
        pdf.savefig(ax.figure,dpi = 300)
        if (close ==1):
            plt.close()
                        
    for i in range(0,4):
        test_flat = plot_array[i][0].values.flatten()
        predicted_flat = plot_array[i][1].values.flatten()
        fig, ax = plt.subplots(figsize = (5, 6))
        plt.scatter(predicted_flat, test_flat)
        ax.set_xlabel('Predicted values')
        ax.set_ylabel('True values')
        line = mlines.Line2D([0, 1], [0, 1],linewidth = 1.0, color='black',linestyle='dashed')
        transform = ax.transAxes
        line.set_transform(transform)
        ax.add_line(line)
        ax.set_xscale('symlog')
        ax.set_yscale('symlog')
        ax.set_title(message+' Flag = '+str(i)+'.')
        plt.savefig('plots/Scatter. '+message+' Flag = '+str(i)+'.jpg',dpi =300)            
        pdf.savefig(ax.figure,dpi = 300)
        if (close ==1):
            plt.close()    
    pdf.close()

In [4]:
def create_dataset(df, df_0, future_flag):
    items = df.item.unique()
    df = df[df['future_flag'] == future_flag]
    df = df.groupby(['rpd', 'item']).sum().reset_index().drop('future_flag', axis = 1)
    for item in items:
        tmp = df[df['item'] == item].drop('item', axis = 1)
        tmp.columns = ['rpd', str(item)]
        df_0 = df_0.merge(tmp, on = 'rpd', how = 'left')
    df_0.set_index('rpd', inplace = True)
    return df_0
def create_dataframes_for_plots(df):
    df_0 = pd.DataFrame(df.rpd.unique())
    df_0.columns = ['rpd']
    df_0.sort_values(by = 'rpd', ascending = True, inplace = True)
    df_flag_0 = create_dataset(df, df_0, 0) # dataset for future_flag = 0 
    df_flag_1 = create_dataset(df, df_0, 1) # dataset for future_flag = 1
    df_flag_2 = create_dataset(df, df_0, 2) # dataset for future_flag = 2 
    df_flag_3 = create_dataset(df, df_0, 3) # dataset for future_flag = 3
    return df_flag_0,df_flag_1,df_flag_2,df_flag_3
def smape_(y_true, y_pred, mean = True):
    smape = 2 * np.sum(np.abs(y_pred-y_true) / np.abs(y_true + y_pred), axis = 0)*100/len(y_pred)
    if mean == True:
        smape = smape.mean()
    return smape

## Creating exp_y,exp_x

In [5]:
def make_Xy(data, n_order=1):
    n_ahead = data.shape[1]
    data_X = pd.concat([
        data.shift(-h).rename(columns=f"{h}-{{}}".format).iloc[:, h:]
        for h in range(n_ahead)
    ], axis=1)

    # get q^h_{:t} -- autoregressive features (known by t-h-1)
    if n_order == 1:
        data_X = data_X.shift(1)

    else:
        data_X = pd.concat([
            data_X.shift(lag).rename(f"{lag}-{{}}".format, axis=1)
            for lag in range(1, n_order + 1)
        ], axis=1)
    # end if

    # get (q^h_{t+s})_{h < s, s=1..4} -- current target (known at t+1, t+2, t+3 and t+4)
    data_y = pd.concat([
        data.shift(-h).rename(columns=f"{h+1}-{{}}".format).iloc[:, :h+1]
        for h in range(n_ahead)
    ], axis=1)

    # return data_X, data_y

    # X and y are already properly shifted and thus the NaNs can just be dropped
    data_X, data_y = data_X.dropna(), data_y.dropna()
    return data_X.align(data_y, join="inner", axis=0)

In [42]:
def create_X_y(df,first_rpd=1,last_rpd=45,predicted_rpd=37):
    """first_rpd: int
        Data in dataset starts with this rpd. (1 for dataset 1-2;108 for dataset 3)
    last_rpd: int
        Data in dataset ends with this rpd. (45 for dataset 1-2;151 for dataset 3)
    predicted_rpd:int
        Algorithm will start predicting values starting from this rpd. (37 for dataset 1-2;143 for dataset 3) """
    df = df.set_index(["item", "rpd", "future_flag"]).sort_index()
    all_items = set(df.index.get_level_values('item'))
    y_predicted = defaultdict()
    exp_X = pd.DataFrame(columns=['item','rpd',' 0-0',' 0-1',' 0-2',' 0-3',' 1-1',' 1-2',' 1-3',' 2-2',' 2-3',' 3-3']) 
    exp_Y = pd.DataFrame(columns=['item','rpd',' 1-0',' 2-0',' 2-1',' 3-0',' 3-1',' 3-2',' 4-0',' 4-1',' 4-2',' 4-3'])  
    y_quantity_item = defaultdict()
    for it in all_items:
        item_ = df.query('item == '+str(it))
        #It was stated that skip all items, for which # of rpd<38  (around 120 from 720 items)
        if item_.groupby('rpd').ngroups!=(last_rpd-first_rpd+1):
            continue
        demand_total_item = item_["quantity_ecd"].unstack(fill_value=0)
        qty_total_item = demand_total_item.unstack("item", fill_value=0.)
        if (item_.groupby('future_flag').ngroups==1):
            qty_total_item[1]=0
            qty_total_item[2]=0
            qty_total_item[3]=0
        else:
            if (item_.groupby('future_flag').ngroups==2):
                qty_total_item[2]=0
                qty_total_item[3]=0
            else:
                if (item_.groupby('future_flag').ngroups==3):
                    qty_total_item[3]=0           
        t = make_Xy(qty_total_item)
        X_quantity_item = t[0].loc[first_rpd:last_rpd,:]
        Y_quantity_item = t[1].loc[first_rpd:last_rpd,:]
        X_quantity_item= X_quantity_item.reset_index()
        X_quantity_item.insert(0, 'item', it)
        Y_quantity_item= Y_quantity_item.reset_index()
        Y_quantity_item.insert(0, 'item', it)
        exp_X = exp_X.append(pd.DataFrame(X_quantity_item.values,columns = ['item','rpd',' 0-0',' 0-1',' 0-2',' 0-3',' 1-1',' 1-2',' 1-3',' 2-2',' 2-3',' 3-3']))
        exp_Y = exp_Y.append(pd.DataFrame(Y_quantity_item.values,columns = ['item','rpd',' 1-0',' 2-0',' 2-1',' 3-0',' 3-1',' 3-2',' 4-0',' 4-1',' 4-2',' 4-3']))
    exp_X = exp_X.applymap(int)
    exp_Y = exp_Y.applymap(int)
    return exp_X,exp_Y

## Fitting functions

In [7]:
def prediction_all_items_fit(exp_X_trunc,exp_Y_trunc,regrr,transformation,first_rpd,last_rpd,predicted_rpd):
    y_predicted = defaultdict()
    exp_Y_trunc = exp_Y_trunc.reindex(columns=['rpd','item',' 1-0',' 2-0',' 3-0',' 4-0',' 2-1',' 3-1',' 3-2',' 4-1',' 4-2',' 4-3'])
    y_true = defaultdict()
    for it in set(exp_X_trunc.item):
        y_predicted[it] = [[],[],[],[],[],[],[],[],[],[]]
        y_true[it] = [[],[],[],[],[],[],[],[],[],[]]
    for i in range(predicted_rpd-4,last_rpd):       
        if (transformation==1):
            scaler = MinMaxScaler() 
        X = exp_X_trunc.loc[exp_X_trunc['rpd'].isin(list(range(first_rpd,i+1)))].iloc[:,2:12]
        for ii in range(1,11):
            regr = regrr
            if (transformation==1):
                scaler_y = MinMaxScaler() ##
            Y = exp_Y_trunc.loc[exp_Y_trunc['rpd'].isin(list(range(first_rpd,i+1)))].iloc[:,ii+1] 
            if (transformation==1):
                Y = scaler_y.fit_transform( np.array([list(map(float,Y))]).transpose()) ##
                X_scaled = scaler.fit_transform(X) ##
            if (transformation==0):
                regr.fit(X,Y)
            if (transformation==1):
                regr.fit(X_scaled,np.array(Y).ravel())
            if (transformation==2):
                regr.fit(np.log1p(X),np.log1p(Y))
            for it in set(exp_X_trunc.item):
                X_test = exp_X_trunc.loc[exp_X_trunc['rpd'].isin(list(range(i+1,i+2)))]
                X_test = X_test.loc[X_test['item']==it].iloc[:,2:12]
                if (X_test.size==0):
                    continue
                if (transformation==0):
                    y_predicted[it][ii-1].append(regr.predict(X_test)[0])
                if (transformation==1):
                    scaled_prediction = regr.predict( scaler.transform(X_test))[0]
                    y_predicted[it][ii-1].append(  scaler_y.inverse_transform(scaled_prediction)[0][0])
                if (transformation==2):
                    y_predicted[it][ii-1].append(  np.expm1 (regr.predict(np.log1p(X_test))[0]))              
                true_ = exp_Y_trunc.loc[exp_Y_trunc['rpd'].isin(list(range(i+1,i+2)))]
                true_ = true_.loc[true_['item']==it].iloc[:,ii+1].values[0]
                y_true[it][ii-1].append(true_)
    for x in y_predicted.keys():
        y_predicted[x] = pd.DataFrame(np.array(y_predicted[x]).clip(min=0).transpose())
        y_true[x] = pd.DataFrame(np.array(y_true[x]).transpose()).applymap(float)
    return y_predicted,y_true

In [8]:
def prediction_item_by_item_fit(exp_X_trunc,exp_Y_trunc,regrr,transformation,first_rpd,last_rpd,predicted_rpd):
    y_predicted = defaultdict()
    exp_Y_trunc = exp_Y_trunc.reindex(columns=['rpd','item',' 1-0',' 2-0',' 3-0',' 4-0',' 2-1',' 3-1',' 3-2',' 4-1',' 4-2',' 4-3'])
    y_true = defaultdict()
    for it in set(exp_X_trunc.item):
        y_predicted[it] = [[],[],[],[],[],[],[],[],[],[]]
        y_true[it] = [[],[],[],[],[],[],[],[],[],[]]
    for i in range(predicted_rpd-4,last_rpd):  
        for ii in range(1,11):
            for it in set(exp_X_trunc.item):
                regr = regrr
                X = exp_X_trunc.loc[exp_X_trunc['rpd'].isin(list(range(first_rpd,i+1)))]
                X = X.loc[X['item']==it].iloc[:,2:12]
                Y = exp_Y_trunc.loc[exp_Y_trunc['rpd'].isin(list(range(first_rpd,i+1)))]
                Y = Y.loc[Y['item']==it].iloc[:,ii+1]
                if (transformation==0):
                    regr.fit(X,Y)          
                    X_test = exp_X_trunc.loc[exp_X_trunc['rpd'].isin(list(range(i+1,i+2)))]
                    X_test = X_test.loc[X_test['item']==it].iloc[:,2:12]
                    if (X_test.size==0):
                        continue
                    y_predicted[it][ii-1].append(regr.predict(X_test)[0])
                if (transformation==1):
                    scaler = MinMaxScaler() 
                    scaler_y = MinMaxScaler()
                    Y = scaler_y.fit_transform( np.array([list(map(float,Y))]).transpose())
                    X_scaled = scaler.fit_transform(X) 
                    regr.fit(X_scaled,np.array(Y).ravel())
                    X_test = exp_X_trunc.loc[exp_X_trunc['rpd'].isin(list(range(i+1,i+2)))]
                    X_test = X_test.loc[X_test['item']==it].iloc[:,2:12]
                    if (X_test.size==0):
                        continue
                    scaled_prediction = regr.predict( scaler.transform(X_test))[0]
                    y_predicted[it][ii-1].append(  scaler_y.inverse_transform(scaled_prediction)[0][0])
                if (transformation==2):
                    regr.fit(np.log1p(X),np.log1p(Y))
                    X_test = exp_X_trunc.loc[exp_X_trunc['rpd'].isin(list(range(i+1,i+2)))]
                    X_test = X_test.loc[X_test['item']==it].iloc[:,2:12]
                    if (X_test.size==0):
                        continue
                    y_predicted[it][ii-1].append(  np.expm1 (regr.predict(np.log1p(X_test))[0]))                    
                true_ = exp_Y_trunc.loc[exp_Y_trunc['rpd'].isin(list(range(i+1,i+2)))]
                true_ = true_.loc[true_['item']==it].iloc[:,ii+1].values[0]
                y_true[it][ii-1].append(true_)
    for x in y_predicted.keys():
        y_predicted[x] = pd.DataFrame(np.array(y_predicted[x]).clip(min=0).transpose())
        y_true[x] = pd.DataFrame(np.array(y_true[x]).transpose()).applymap(float)
    return y_predicted,y_true

## Prediction

In [19]:
def prediction(exp_X,exp_Y,regr,fitting=0,transformation=0,first_rpd = 1, last_rpd = 45, predicted_rpd = 37,name=""):
    """ Predicts values using matrix feeding
    exp_X : DataFrame of features
        Should be generated using create_X_y function.
    exp_Y: DataFrame of targets
        Should be generated using create_X_y function.
    regr: 
        Regressor with sklearn api.
    fitting: int
        Type of fitting: 0 - fitting item by item, 1 - fit all items at once.
    transformation: int
        Type of transformation: 0 - no transformation, 1 - MinMax scaling, 2 - log transformation.
    name: string
        Name of the output file and. The default one will combine parameters.
    first_rpd: int
        Data in dataset starts with this rpd. (1 for dataset 1-2;108 for dataset 3)
    last_rpd: int
        Data in dataset ends with this rpd. (45 for dataset 1-2;151 for dataset 3)
    predicted_rpd:int
        Algorithm will start predict values starting from this rpd. (37 for dataset 1-2;143 for dataset 3) """
    if (name==""):
        name = str(regr)[:10]+str(fitting)+str(transformation)
    # Predicts values
    if (fitting==0):
        y_predicted, y_true= prediction_item_by_item_fit(exp_X,exp_Y,regr,transformation,first_rpd,last_rpd,predicted_rpd)
    if (fitting==1):
        y_predicted, y_true= prediction_all_items_fit(exp_X,exp_Y,regr,transformation,first_rpd,last_rpd,predicted_rpd)
    # Prints to csv
    true_,pred_  = print_to_csv(y_predicted,y_true,last_rpd,predicted_rpd,name)
    # Plotting
    true_0,true_1,true_2,true_3 = create_dataframes_for_plots(true_)
    pred_0,pred_1,pred_2,pred_3= create_dataframes_for_plots(pred_)
    plot_array = [(true_0,pred_0),(true_1,pred_1),(true_2,pred_2),(true_3,pred_3)]
    plot_hist(plot_array, name)

### Example

#### Dataset 1

In [40]:
df = pd.read_csv("input/demand_encode_out.csv", header=0, index_col=None)
exp_X,exp_Y = create_X_y(df,1,45,37)

In [41]:
prediction(exp_X,exp_Y,MLPRegressor(activation= 'tanh', hidden_layer_sizes= (20,), solver= 'lbfgs'),
           fitting = 0,transformation = 1,
           first_rpd = 1,last_rpd =45,predicted_rpd = 37,
           name = "NN;item_by_item_fit;minmax;dataset1")

#### Dataset 3

In [ ]:
df = pd.read_csv("input/demand_out_encoded_stage_3.csv", header=0, index_col=None)
exp_X,exp_Y = create_X_y(df,108,151,143)

In [ ]:
prediction(exp_X,exp_Y,MLPRegressor(activation= 'tanh', hidden_layer_sizes= (20,), solver= 'lbfgs'),
           fitting = 1,transformation = 2,
           first_rpd = 108,last_rpd =151,predicted_rpd = 143,
           name = "NN;all_fit;log;dataset3")